#3. *k*-Nearest Neighbors
In the third ML exercise session, we will explore the *k*-Nearest Neighbors classification algorithm.

Again, we start by executing the import statements to load the necessary libraries. As a review, the library named **numpy** provides support for handling matrices and multi-dimensional arrays efficiently. It is often used with the alias `np` (aliases can be specified using `as`). The **operator** library supports using common operators as functions. Third, **matplotlib** offers various functionalities to draw line plots, scatter plots, histograms, and more. It is commonly used with the alias `plt`.

In [ ]:
›import numpy as np
import operator
import matplotlib.pyplot as plt

Next, mount your Google Drive to access data files. To practice the k-NN classification algorithm, we need the following files:
* datingTestSet.txt
* digits.zip

Data files: https://github.com/charmgil/DSCC2023

All the practice code used during the camp assumes that the data files are located under datasets/DSCC/ in each individual's Google Drive.

Once the files are copied, execute the following two lines of code. Then, log in with your Google account using the link that appears. After logging in, enter the pass phrase shown on the screen in Colab.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 3.1. Load a data file
Once you have mounted Google Drive, you can execute the following function to read the file and load the data.


In [ ]:
def file2matrix(filename):
  file = open(filename)
  n = len(file.readlines())

  return_matrix = np.zeros((n, 3))
  class_labels = []
  file = open(filename)
  index = 0
  for line in file.readlines():
    line = line.strip()
    tokens = line.split('\t')
    return_matrix[index, :] = tokens[0:3]
    class_labels.append(tokens[-1])
    index += 1

  return return_matrix, class_labels

Now, let us execute the following two lines of code. Make sure that the argument for `file2matrix()` corresponds to the location of the `datingTestSet.txt` file on your Google Drive.

In [ ]:
X, y = file2matrix('drive/MyDrive/datasets/DSCC/datingTestSet.txt')

print(y)
print(X)

`datingTestSet.txt` contains some content from the data where one user of a dating site has indicated her/his preferences for 1,000 other user profiles.

Through the label y, which represents the preferences, you can see the following items:

* `didn't like` indicates that the user did not like the other person at all.
* `small doses` indicates that the user liked the other person a little.
* `large doses` indicates that the user liked the other person a lot.

Each user's profile `X`, which denotes the input features, contains the following three variables:
* First column: annual airline mileage (miles)
* Second column: percentage of time spent on playing games annually
* Third column: annual ice cream consumption (liters)

### 3.2. Data Normalization


Just like when dealing with the *k*-means clustering algorithm, it is essential to go through the normalization process before providing data to the *k*-NN algorithm. For this purpose, we have prepared a normalization routine consisting of three functions today: `apply_normalizer()`, `normalize_minmax()`, `and normalize_meanstd()`.






In [ ]:
def apply_normalizer(dataset, offset, divisor):
  dataset_normalized = np.zeros(dataset.shape)
  N = dataset.shape[0]
  dataset_normalized = dataset - np.tile(offset, (N,1))
  dataset_normalized = dataset_normalized / np.tile(divisor, (N,1))

  return dataset_normalized


def normalize_minmax(dataset):
  minval = dataset.min(0)
  maxval = dataset.max(0)

#   dataset_normalized = np.zeros(dataset.shape)
#   N = dataset.shape[0]
#   dataset_normalized = dataset - np.tile(minval, (N,1))
#   dataset_normalized = dataset_normalized / np.tile(maxval-minval, (N,1))

  dataset_normalized = apply_normalizer(dataset, minval, maxval-minval)

  return dataset_normalized, minval, maxval-minval


def normalize_meanstd(dataset):
  meanval = dataset.mean(0)
  stdval = dataset.std(0)

#   dataset_normalized = np.zeros(dataset.shape)
#   N = dataset.shape[0]
#   dataset_normalized = dataset - np.tile(meanval, (N,1))
#   dataset_normalized = dataset_normalized / np.tile(stdval, (N,1))

  dataset_normalized = apply_normalizer(dataset, meanval, stdval)

  return dataset_normalized, meanval, stdval

## 3.3. Implement *k*-Nearest Neighbors Algorithm (kNN)

Now it is time to implement the kNN classifier. Refer to the content from the lectures and the provided code below to write your first prediction program.

In [ ]:
def knn(x_test, X_train, labels, k):

  N_train = X_train.shape[0]

  # 1: Measure the distance from x_test for all data points (rows) in X_train
  diff_mat = np.tile(x_test, (N_train, 1)) - X_train
  sq_diff_mat = diff_mat ** 2
  sq_distances = sq_diff_mat.sum(axis=1)
  distances = sq_distances ** 0.5

  # 2: We find the k nearest X_train data points from x_test and construct the "neighbor set" I
  sorted_dist_indicies = distances.argsort()

  # 3: We make a prediction for x_test by following the majority label in the set I
  class_count = {}
  for i in range(k):
    vote = labels[sorted_dist_indicies[i]]
    class_count[vote] = class_count.get(vote, 0) + 1
  sorted_class_count = sorted(class_count.items(), key=operator.itemgetter(1), reverse=True)

  # 4: return the result
  result = sorted_class_count[0][0]
  # print(class_count)
  return result

Q: What does `np.tile()` do in line 4? What does the output `diff_mat` contain?

Hint: As specified earlier, `np` refers to **numpy**. The `tile` function belongs to the **numpy** library, and you can find its usage through the following link:
https://docs.scipy.org/doc/numpy/reference/generated/numpy.tile.html


The developers of **numpy** provide documentation for all members of **numpy**, including the `tile` function. You can check it out through the following link.
https://docs.scipy.org/doc/numpy/reference/routines.html

### 3.4. Test the classifier
Let us examine the prediction accuracy on the prepared data. The test will be conducted by separating and holding out 20% of the data as **test data** and applying it to *k*NN.






In [ ]:
# Reload the dataset
X, y = file2matrix('drive/MyDrive/datasets/DSCC/datingTestSet.txt')

holdout_ratio = .2

N = X.shape[0] # number of instances
N_ts = int(N*holdout_ratio) # number of test instances
N_tr = N - N_ts # number of training instances


# Split the dataset
X_tr = X[0:N_tr,:]
y_tr = y[0:N_tr]

X_ts = X[N_tr:,:]
y_ts = y[N_tr:]

# Uncomment the following lines, if you want to see the data contents
# print(N_tr)
# print(N_ts)

# print(X_tr.shape)
# print(X_ts.shape)
# print(X_tr)
# print(X_ts)

# print(y_tr)
# print(y_ts)


# Normalize the dataset
X_normalized_tr, off, div = normalize_minmax(X_tr)
X_normalized_ts = apply_normalizer(X_ts, off, div)


# Apply the classifier
n_errors = 0
y_pred_ts = []
for i in range(N_ts):
  y_pred_ts.append(knn(X_normalized_ts[i], X_normalized_tr, y_tr, 3))
  if(y_pred_ts[i] != y_ts[i]):
    n_errors += 1

print("the accuracy is: %f" % (1 - n_errors/float(N_ts)))
print("the error rate is: %f" % (n_errors/float(N_ts)))


print("\n---- (Y_true, Y_pred) pairs ----")
print(*list(zip(y_ts, y_pred_ts)), sep="\n")

### 3.5. An Application
Finally, using the loaded dataset and *k*NN, we will create a program that predicts matching scores based on user input values. That is, when you run the program, you can test it by entering answers to the questions in the input box, and you will be able to see real-time prediction results.

In [ ]:
def classifyPerson():
  resultList = ['not at all','in small doses', 'in large doses']

  percentTats = float(input("percentage of time spent playing video games? "))
  ffMiles = float(input("frequent flier miles earned per year? "))
  iceCream = float(input("liters of ice cream consumed per year? "))

  X_hist, y_hist = file2matrix('drive/MyDrive/datasets/DSCC/datingTestSet.txt')

  X_hist_normalized, off, div = normalize_minmax(X_hist)
  X_test = np.array([ffMiles, percentTats, iceCream])
  classifierResult = knn((X_test - off)/div, X_hist_normalized, y_hist, 3)
  print("You will probably like this person: ", classifierResult)

classifyPerson()

<hr/>

## Assignment: Analyze Results
Here we will use the **matplotlib** library to take a quick look at the distribution of the data.

First, we reload the data.

In [ ]:
X, y = file2matrix('drive/MyDrive/datasets/DSCC/datingTestSet.txt')

y_int = []
for i in range(N):
  if y[i] == 'largeDoses':
    y_int.append(3)
  elif y[i] == 'smallDoses':
    y_int.append(2)
  else:
    y_int.append(1)

In the following visualization code, we use the 2nd and 3rd columns (index 1 and 2) of the input data to examine the distribution of the output labels (`y` values).

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

fig = plt.figure()
ax = fig.add_subplot(111)
# ax.scatter(X[:,1], X[:,2])
ax.scatter(X[:,1], X[:,2], 15*np.array(y_int), 15*np.array(y_int))
plt.show()

In the second visualization code, we use the 1st and 2nd columns (index 0 and 1) of the input data to examine the distribution of the output labels (`y` values).

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(X[:,0], X[:,1], 15*np.array(y_int), 15*np.array(y_int))
plt.show()

Q: If we choose better input features for decision-making, among the 1st, 2nd, and 3rd columns, which ones would we select?

<hr/>

## Take-home Assignment: Classifying hand-written digits

Let us take a look at a new dataset. Before executing the code below, please download and extract the `digits.zip` file, then upload the dataset to your Google Drive.

In the following example, we assume that the handwritten digits dataset (numbers `0-9` written manually) is located in `drive/MyDrive/datasets/DSCC/digits/trainingDigits` and `testDigits` directories. These directories contain text-formatted binary images. For instance, the file `trainingDigits/0_1.txt` contains the following text (**Do you see a large 0 drawn with 0s and 1s?**):

```
00000000000111110000000000000000
00000000001111111000000000000000
00000000011111111100000000000000
00000000111111111110000000000000
00000001111111111111000000000000
00000011111110111111100000000000
00000011111100011111110000000000
00000011111100001111110000000000
00000111111100000111111000000000
00000111111100000011111000000000
00000011111100000001111110000000
00000111111100000000111111000000
00000111111000000000011111000000
00000111111000000000011111100000
00000111111000000000011111100000
00000111111000000000001111100000
00000111111000000000001111100000
00000111111000000000001111100000
00000111111000000000001111100000
00000111111000000000001111100000
00000011111000000000001111100000
00000011111100000000011111100000
00000011111100000000111111000000
00000001111110000000111111100000
00000000111110000001111111000000
00000000111110000011111110000000
00000000111111000111111100000000
00000000111111111111111000000000
00000000111111111111110000000000
00000000011111111111100000000000
00000000001111111111000000000000
00000000000111111110000000000000
```

Additionally, the number before the underscore (`_`) in the filename represents the label, which corresponds to numbers `0` to `9`. For example, in `0_1.txt`, the `0` before the underscore represents the label.

By writing and executing the code in your Colab Notebook, you will be able to observe the process of reading and processing the text files representing the digit images one by one. Furthermore, at the end of the code execution, you can check the overall accuracy and error rate.

In [ ]:
import os

K = 5

IMG_WIDTH = 32
IMG_HEIGHT = 32

def img2vector(filename):
  vect = np.zeros((1, 1024))
  file = open(filename)
  for i in range(IMG_HEIGHT):
    line = file.readline()
    for j in range(IMG_WIDTH):
      vect[0, 32*i+j] = int(line[j])
  return vect

def classifyHandwrittenDigit():
  dirname_str = 'drive/MyDrive/datasets/MLPracticum/digits/trainingDigits'
  filelist_tr = os.listdir(dirname_str)
  N_tr = len(filelist_tr)
  X_tr = np.zeros((N_tr, 1024))
  y_tr = []

  # training data
  for i in range(N_tr):
    if i%100 == 0:
      print('(train) Busy...', i, '/', N_tr)

    filename_str = filelist_tr[i].split('.')[0]
    label = int(filename_str.split('_')[0])
    y_tr.append(label)
    X_tr[i,:] = img2vector('%s/%s' % (dirname_str, filelist_tr[i]))


  n_errors = 0

  # test data
  dirname_str = 'drive/MyDrive/datasets/MLPracticum/digits/testDigits'
  filelist_ts = os.listdir(dirname_str)
  N_ts = len(filelist_ts)
  for i in range(N_ts):
    if i%100 == 0:
      print('(test) Busy...', i, '/', N_ts)

    filename_str = filelist_ts[i].split('.')[0]
    label = int(filename_str.split('_')[0])
    X_ts = img2vector('%s/%s' % (dirname_str, filelist_ts[i]))
    classifierResult = knn(X_ts, X_tr, y_tr, K)

    print(type(classifierResult))

    print("the classifier came back with: %d, the real answer is: %d" % (classifierResult, label))
    if (classifierResult != label): n_errors += 1.0
  print("\nthe total number of errors is: %d" % n_errors)
  print("\nthe accuracy is: %f" % (1 - n_errors/float(N_ts)))
  print("\nthe error rate is: %f" % (n_errors/float(N_ts)))

classifyHandwrittenDigit()

### References
* Harrington. *Machine Learning in Action*. Manning Publications Co. 2012.